# Project 3 - Part 4

Author: Nicholas Orgel
    
Date: 03/15/2023

## Questions to Answer
The stakeholder's first question is: does the MPAA rating of a movie (G/PG/PG-13/R) affect how much revenue the movie generates?

They want you to perform a statistical test to get a mathematically-supported answer.
They want you to report if you found a significant difference between ratings.
If so, what was the p-value of your analysis?
And which rating earns the most revenue?
They want you to prepare a visualization that supports your finding.
It is then up to you to think of 2 additional hypotheses to test that your stakeholder may want to know.

**Some example hypotheses you could test:**

- Do movies that are over 2.5 hours long earn more revenue than movies that are 1.5 hours long (or less)?
- Do movies released in 2020 earn less revenue than movies released in 2018?
- How do the years compare for movie ratings?
- Do some movie genres earn more revenue than others?
- Are some genres higher rated than others?
- etc.